In [1]:
!pip install -U opencv-python tensorflow scikit-learn pandas matplotlib tensorflow_datasets

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 573.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 154.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 225.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 143.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 274.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 180.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 337.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 277.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 342.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 175.9 MB/s eta 0:00:00
   ━━━━

# IMPORTATION DES LIBRAIRIES

In [2]:
import numpy as np
import pandas as pd
import os
import sklearn
import tensorflow as tf
from tensorflow import keras
import cv2
import matplotlib.pyplot as plt
import tensorflow_datasets.public_api as tfds
import requests
import zipfile
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

2025-08-13 08:08:36.626565: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755072516.640318     409 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755072516.644866     409 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755072516.655804     409 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755072516.655817     409 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755072516.655819     409 computation_placer.cc:177] computation placer alr

# Chargement du dataset

In [3]:
def telecharger_dezip(url, chemin_sauv="plant_village_dataset.zip", extract_path="."):
    print(" Début du téléchargement")
    try:
        response=requests.get(url, stream=True)
        response.raise_for_status()

        #Taille totale du fichier pour la barre de progression
        total_size=int(response.headers.get('content-length',0))
        block_size=1064
        bar_progression = tqdm(total=total_size, unit='iB', unit_scale=True)

        #Téléchargement
        with open(chemin_sauv, 'wb') as file:
            for data in response.iter_content(block_size):
                bar_progression.update(len(data))
                file.write(data)
        bar_progression.close()

        if total_size != 0 and bar_progression.n != total_size:
            print("ERREUR, quelque chose s'est mal passé pendant le téléchargement.")
            return

        print(f"Téléchargement terminé. Fichier sauvegardé sous : {chemin_sauv}")

        # Créer le dossier d'extraction s'il n'existe pas
        if not os.path.exists(extract_path):
            os.makedirs(extract_path)

        # Décompresser le fichier ZIP
        print(f"Décompression du fichier dans le dossier : {extract_path}")
        with zipfile.ZipFile(chemin_sauv, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

        print("Décompression terminée.")

        # Optionnel : Supprimer le fichier .zip après extraction pour économiser de l'espace
        print(f"Suppression du fichier {chemin_sauv}...")
        os.remove(chemin_sauv)
        print("Opération terminée avec succès !")

    except requests.exceptions.RequestException as e:
        print(f"Une erreur de réseau est survenue: {e}")
    except zipfile.BadZipFile:
        print("Erreur: Le fichier téléchargé n'est pas un fichier ZIP valide.")
    except Exception as e:
        print(f"Une erreur inattendue est survenue: {e}")

In [4]:
URL = "https://data.mendeley.com/datasets/tywbtsjrjv/1/files/b4e3a32f-c0bd-4060-81e9-6144231f2520/file_downloaded"

In [5]:
extract_folder = "plant_village_dataset"

In [6]:
telecharger_dezip(URL, "PlantVillage.zip", extract_folder)

 Début du téléchargement


100%|██████████| 949M/949M [00:35<00:00, 26.7MiB/s]  


Téléchargement terminé. Fichier sauvegardé sous : PlantVillage.zip
Décompression du fichier dans le dossier : plant_village_dataset
Décompression terminée.
Suppression du fichier PlantVillage.zip...
Opération terminée avec succès !


In [7]:
path="/workspace/plant_village_dataset/Plant_leave_diseases_dataset_with_augmentation"

In [8]:
IMG_SIZE = (50, 50)
BATCH_SIZE = 32

In [9]:
data_gen=ImageDataGenerator(rescale=1./255)

In [10]:
data=data_gen.flow_from_directory(
    path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

Found 61486 images belonging to 39 classes.


# MODELISATION

## Personnalisé CNN

In [11]:
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import psutil
import torch.nn.functional as F
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [12]:
# --------- 1. Préparer les données ---------
filepaths = []
labels = []
folds = os.listdir(path)
for fold in folds:
    f_path = os.path.join(path, fold)
    if not os.path.isdir(f_path):
        continue
    for file in os.listdir(f_path):
        filepaths.append(os.path.join(f_path, file))
        labels.append(fold)

df = pd.DataFrame({'filepaths': filepaths, 'labels': labels})
print(f"Total des images trouvées : {len(df)}")

Total des images trouvées : 61486


In [13]:
# Split 80/20 avec stratification
train_df, test_df = train_test_split(
    df,
    test_size=0.1,
    random_state=42,
    stratify=df['labels']
)
train_df, val_df = train_test_split(
    train_df,
    test_size=0.2,
    random_state=42,
    stratify=train_df['labels']
)

In [14]:
# Mapping des classes en indices
class_names = sorted(df['labels'].unique())
class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}
num_classes = len(class_names)

In [15]:
# --------- 2. Dataset personnalisé ---------
class CustomImageDataset(Dataset):
    def __init__(self, df, class_to_idx, transform=None):
        self.df = df.reset_index(drop=True)
        self.class_to_idx = class_to_idx
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'filepaths']
        label_name = self.df.loc[idx, 'labels']
        label = self.class_to_idx[label_name]
        
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

In [16]:
# --------- 3. Data augmentation et loaders ---------
train_transforms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]) # EfficientNet normalization
])



val_transforms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [17]:
train_dataset = CustomImageDataset(train_df, class_to_idx, transform=train_transforms)
val_dataset = CustomImageDataset(val_df, class_to_idx, transform=val_transforms)
test_dataset = CustomImageDataset(test_df, class_to_idx, transform=val_transforms)

In [18]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
class DeepCNN(nn.Module):
    def __init__(self, num_classes):
        super(DeepCNN, self).__init__()
        
        self.conv1 = nn.Conav2d(3, 32, kernel_size=3, padding=0)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=0)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=0)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=0)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        # Calcul automatique de la taille après convolution/pooling
        self.flatten_dim = self._get_flatten_dim()
        
        self.fc1 = nn.Linear(self.flatten_dim, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, num_classes)
        
    def _get_flatten_dim(self):
        with torch.no_grad():
            x = torch.zeros(1, 3, 50, 50)  # batch=1, channels=3, H=50, W=50
            x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
            x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
            return x.view(1, -1).shape[1]

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool2(x)
        
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
model_deepcnn = DeepCNN(num_classes=len(class_names)).to(device)

In [23]:
# Fonction de perte (équivalent à categorical_crossentropy)
criterion = nn.CrossEntropyLoss()

# Optimiseur (équivalent à optimizer='adam')
optimizer = optim.Adam(model_deepcnn.parameters(), lr=0.001)

In [24]:
best_model_path = "/workspace/models/best_model_deepcnn.h5"

In [25]:
best_val_acc = 0.0
epochs_no_improve = 0
patience = 5  

In [26]:
EPOCHS = 20

In [27]:
for epoch in range(EPOCHS):
    # --- TRAIN ---
    model_deepcnn.train()
    train_loss, train_correct = 0.0, 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_deepcnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()

    train_loss /= len(train_loader.dataset)
    train_acc = train_correct / len(train_loader.dataset)

    # --- VALIDATION ---
    model_deepcnn.eval()
    val_loss, val_correct = 0.0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model_deepcnn(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            val_correct += (outputs.argmax(1) == labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_acc = val_correct / len(val_loader.dataset)

    print(f"Epoch [{epoch+1}/{EPOCHS}] - "
          f"Train loss: {train_loss:.4f}, acc: {train_acc:.4f} | "
          f"Val loss: {val_loss:.4f}, acc: {val_acc:.4f}")

    # --- Early stopping & checkpoint ---
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model_deepcnn.state_dict(), best_model_path)
        print(">> Nouveau meilleur modèle sauvegardé")
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print(">> Early stopping déclenché")
            break

Epoch [1/20] - Train loss: 2.4761, acc: 0.3211 | Val loss: 1.4572, acc: 0.5517
>> Nouveau meilleur modèle sauvegardé
Epoch [2/20] - Train loss: 1.7744, acc: 0.4891 | Val loss: 0.9525, acc: 0.6987
>> Nouveau meilleur modèle sauvegardé
Epoch [3/20] - Train loss: 1.4546, acc: 0.5730 | Val loss: 0.7672, acc: 0.7551
>> Nouveau meilleur modèle sauvegardé
Epoch [4/20] - Train loss: 1.2781, acc: 0.6216 | Val loss: 0.5962, acc: 0.8037
>> Nouveau meilleur modèle sauvegardé
Epoch [5/20] - Train loss: 1.1465, acc: 0.6588 | Val loss: 0.5249, acc: 0.8345
>> Nouveau meilleur modèle sauvegardé
Epoch [6/20] - Train loss: 1.0538, acc: 0.6847 | Val loss: 0.4221, acc: 0.8657
>> Nouveau meilleur modèle sauvegardé
Epoch [7/20] - Train loss: 0.9795, acc: 0.7079 | Val loss: 0.4121, acc: 0.8666
>> Nouveau meilleur modèle sauvegardé
Epoch [8/20] - Train loss: 0.9430, acc: 0.7177 | Val loss: 0.3651, acc: 0.8838
>> Nouveau meilleur modèle sauvegardé
Epoch [9/20] - Train loss: 0.9043, acc: 0.7293 | Val loss: 0.307

### Evaluation

In [29]:
import time

In [30]:
# Charger le meilleur modèle sauvegardé
model_deepcnn.load_state_dict(torch.load(best_model_path, map_location=device))
model_deepcnn.to(device)
model_deepcnn.eval()

all_preds, all_labels = [], []
start_time = time.time()

In [31]:
with torch.no_grad():
    for batch_idx, (inputs, labels) in enumerate(test_loader):
        batch_start = time.time()

        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model_deepcnn(inputs)
        preds = torch.argmax(outputs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        # Utilisation CPU/RAM
        cpu_usage = psutil.cpu_percent(interval=None)
        ram = psutil.virtual_memory()

        # Utilisation GPU (si dispo)
        if torch.cuda.is_available():
            gpu_mem = torch.cuda.memory_allocated(device) / 1024**2  # en MB
        else:
            gpu_mem = 0.0

        print(f"[Batch {batch_idx+1}] Temps: {time.time()-batch_start:.2f}s | "
              f"CPU: {cpu_usage:.1f}% | RAM: {ram.used/1024**3:.2f}GB | GPU: {gpu_mem:.2f}MB")

end_time = time.time()

[Batch 1] Temps: 0.00s | CPU: 6.8% | RAM: 193.63GB | GPU: 38.59MB
[Batch 2] Temps: 0.00s | CPU: 3.8% | RAM: 193.60GB | GPU: 38.59MB
[Batch 3] Temps: 0.00s | CPU: 4.2% | RAM: 193.60GB | GPU: 38.59MB
[Batch 4] Temps: 0.00s | CPU: 2.5% | RAM: 193.55GB | GPU: 38.59MB
[Batch 5] Temps: 0.00s | CPU: 15.0% | RAM: 192.79GB | GPU: 38.59MB
[Batch 6] Temps: 0.00s | CPU: 52.1% | RAM: 192.75GB | GPU: 38.59MB
[Batch 7] Temps: 0.00s | CPU: 4.8% | RAM: 192.22GB | GPU: 38.59MB
[Batch 8] Temps: 0.00s | CPU: 5.4% | RAM: 192.19GB | GPU: 38.59MB
[Batch 9] Temps: 0.00s | CPU: 4.5% | RAM: 191.40GB | GPU: 38.59MB
[Batch 10] Temps: 0.00s | CPU: 6.2% | RAM: 191.35GB | GPU: 38.59MB
[Batch 11] Temps: 0.00s | CPU: 5.7% | RAM: 190.96GB | GPU: 38.59MB
[Batch 12] Temps: 0.00s | CPU: 7.4% | RAM: 190.94GB | GPU: 38.59MB
[Batch 13] Temps: 0.00s | CPU: 5.6% | RAM: 190.40GB | GPU: 38.59MB
[Batch 14] Temps: 0.00s | CPU: 8.3% | RAM: 190.37GB | GPU: 38.59MB
[Batch 15] Temps: 0.00s | CPU: 8.6% | RAM: 190.02GB | GPU: 38.59MB
[B

In [32]:
total_time = end_time - start_time
print(f"\nTemps Test Total: {total_time:.2f} sec")
print(f"Throughput: {len(test_dataset) / total_time:.2f} images/sec")


Temps Test Total: 10.64 sec
Throughput: 577.85 images/sec


In [33]:
# Rapport complet
print("=== Test Set Evaluation (Deep CNN) ===")
print(classification_report(all_labels, all_preds, target_names=class_names))

=== Test Set Evaluation (Deep CNN) ===
                                               precision    recall  f1-score   support

                           Apple___Apple_scab       0.91      0.75      0.82       100
                            Apple___Black_rot       0.91      0.97      0.94       100
                     Apple___Cedar_apple_rust       0.98      0.92      0.95       100
                              Apple___healthy       0.94      0.88      0.91       164
                    Background_without_leaves       0.91      0.96      0.94       114
                          Blueberry___healthy       0.84      1.00      0.91       150
                      Cherry___Powdery_mildew       0.97      0.96      0.97       105
                             Cherry___healthy       0.92      0.98      0.95       100
   Corn___Cercospora_leaf_spot Gray_leaf_spot       0.90      0.88      0.89       100
                           Corn___Common_rust       1.00      0.99      1.00       119
   